# Анализ продаж и прибыльности (Superstore)

## Описание проекта
Данный проект посвящён анализу продаж и прибыльности розничного магазина на основе датасета Superstore.
Цель анализа — выявить ключевые факторы, влияющие на прибыль, а также определить проблемные зоны и точки роста бизнеса.

Анализ выполнен с использованием:
- SQL (предобработка данных, представление v_orders)
- Python (pandas, Jupyter Notebook)

---

## Цели анализа
- Оценить общие показатели продаж и прибыли
- Проанализировать эффективность категорий, сегментов и регионов
- Изучить влияние скидок на маржинальность
- Сформулировать бизнес-инсайты и рекомендации

---

## Структура анализа
1. Общие метрики и обзор данных
2. Анализ по категориям и подкатегориям
3. Углублённый анализ:
   - Сегменты клиентов
   - Скидки и их влияние на прибыль
   - Региональная эффективность
4. Ключевые инсайты и рекомендации бизнесу

---

## Используемые данные
- Источник: Superstore Dataset
- Количество строк: ~10 000 заказов
- Основные поля: дата заказа, регион, сегмент, категория, продажи, прибыль, скидка

---

## Результат
В ходе анализа выявлены ключевые факторы снижения маржинальности, а также сформированы практические рекомендации по оптимизации ценовой и дисконтной политики.

In [406]:
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
engine = create_engine(
    "postgresql+psycopg2://postgres:*********localhost:5******/superstore_db"
)

In [408]:
df = pd.read_sql("SELECT * FROM v_orders;", engine)

In [409]:
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   order_id       9994 non-null   object 
 1   order_date     9994 non-null   object 
 2   order_year     9994 non-null   float64
 3   order_month    9994 non-null   float64
 4   ship_date      9994 non-null   object 
 5   region         9994 non-null   object 
 6   category       9994 non-null   object 
 7   sub_category   9994 non-null   object 
 8   sales          9994 non-null   float64
 9   profit         9994 non-null   float64
 10  discount       9994 non-null   float64
 11  quantity       9994 non-null   int64  
 12  profit_margin  9994 non-null   float64
 13  segment        9994 non-null   object 
dtypes: float64(6), int64(1), object(7)
memory usage: 1.1+ MB


In [410]:
df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce')
df['ship_date'] = pd.to_datetime(df['ship_date'], errors='coerce')

In [411]:
df[['order_date', 'ship_date']].isna().sum()

order_date    0
ship_date     0
dtype: int64

In [412]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   order_id       9994 non-null   object        
 1   order_date     9994 non-null   datetime64[ns]
 2   order_year     9994 non-null   float64       
 3   order_month    9994 non-null   float64       
 4   ship_date      9994 non-null   datetime64[ns]
 5   region         9994 non-null   object        
 6   category       9994 non-null   object        
 7   sub_category   9994 non-null   object        
 8   sales          9994 non-null   float64       
 9   profit         9994 non-null   float64       
 10  discount       9994 non-null   float64       
 11  quantity       9994 non-null   int64         
 12  profit_margin  9994 non-null   float64       
 13  segment        9994 non-null   object        
dtypes: datetime64[ns](2), float64(6), int64(1), object(5)
memory usage: 1.1+

In [413]:
total_sales = df['sales'].sum()
total_profit = df['profit'].sum()
avg_margin = total_profit / total_sales * 100

total_orders = df['order_id'].nunique()

total_sales, total_profit, avg_margin, total_orders
pd.DataFrame({
    'metric': ['Total Sales', 'Total Profit', 'Avg Margin %', 'Total Orders'],
    'value': [
        round(total_sales, 2),
        round(total_profit, 2),
        round(avg_margin, 2),
        total_orders
    ]
})



,metric,value
0,Total Sales,2297201.07
1,Total Profit,286397.79
2,Avg Margin %,12.47
3,Total Orders,5009.00


In [414]:
category_summary = (
    df
    .groupby('category')
    .agg(
        sales=('sales', 'sum'),
        profit=('profit', 'sum'),
        orders=('order_id', 'nunique')
    )
    .reset_index()
)

category_summary['profit_margin'] = (
    category_summary['profit'] / category_summary['sales'] * 100
)

category_summary.sort_values('profit', ascending=False)

,category,sales,profit,orders,profit_margin
2,Technology,836154.10,145455.66,1544,17.395796
1,Office Supplies,719046.99,122490.88,3742,17.035170
0,Furniture,741999.98,18451.25,1764,2.486691


In [415]:
subcat_summary = (
    df
    .groupby(['category', 'sub_category'])
    .agg(
        sales=('sales', 'sum'),
        profit=('profit', 'sum'),
        orders=('order_id', 'nunique')
    )
    .reset_index()
)

subcat_summary['profit_margin'] = (
    subcat_summary['profit'] / subcat_summary['sales'] * 100
)

subcat_summary.sort_values('profit')

,category,sub_category,sales,profit,orders,profit_margin
3,Furniture,Tables,206965.68,-17725.59,307,-8.564507
0,Furniture,Bookcases,114880.05,-3472.56,224,-3.022770
12,Office Supplies,Supplies,46673.52,-1188.99,187,-2.547462
8,Office Supplies,Fasteners,3024.25,949.53,215,31.397206
15,Technology,Machines,189238.68,3384.73,112,1.788604
9,Office Supplies,Labels,12486.30,5546.18,346,44.418122
5,Office Supplies,Art,27118.80,6527.96,731,24.071714
7,Office Supplies,Envelopes,16476.38,6964.10,249,42.267173
2,Furniture,Furnishings,91705.12,13059.25,877,14.240481
4,Office Supplies,Appliances,107532.14,18138.07,451,16.867580


In [416]:
subcat_summary[
    (subcat_summary['sales'] > 100000) &
    (subcat_summary['profit'] < 0)
].sort_values('sales', ascending=False)

,category,sub_category,sales,profit,orders,profit_margin
3,Furniture,Tables,206965.68,-17725.59,307,-8.564507
0,Furniture,Bookcases,114880.05,-3472.56,224,-3.022770


In [417]:
segment_metrics = (
    df
    .groupby('segment')
    .agg(
        Sales=('sales', 'sum'),
        Profit=('profit', 'sum'),
        Orders=('order_id', 'nunique')
    )
    .assign(
        Profit_Margin=lambda x: x['Profit'] / x['Sales']
    )
    .sort_values('Profit', ascending=False)
)

segment_metrics

,Sales,Profit,Orders,Profit_Margin
segment,,,,
Consumer,1161401.34,134119.33,2586,0.115481
Corporate,706146.44,91979.45,1514,0.130255
Home Office,429653.29,60299.01,909,0.140343


In [418]:
segment_share = segment_metrics.copy()

segment_share['Sales_Share'] = (
    segment_share['Sales'] / segment_share['Sales'].sum()
)

segment_share['Profit_Share'] = (
    segment_share['Profit'] / segment_share['Profit'].sum()
)

segment_share

,Sales,Profit,Orders,Profit_Margin,Sales_Share,Profit_Share
segment,,,,,,
Consumer,1161401.34,134119.33,2586,0.115481,0.505572,0.468297
Corporate,706146.44,91979.45,1514,0.130255,0.307394,0.321160
Home Office,429653.29,60299.01,909,0.140343,0.187033,0.210543


In [419]:
df[['discount', 'profit']].corr()

,discount,profit
discount,1.000000,-0.219488
profit,-0.219488,1.000000


In [420]:
df['discount_bin'] = pd.cut(
    df['discount'],
    bins=[-0.01, 0, 0.2, 0.4, 0.6, 1],
    labels=['0%', '1–20%', '21–40%', '41–60%', '60%+']
)

discount_profit = (
    df
    .groupby('discount_bin', observed=True)
    .agg(
        Sales=('sales', 'sum'),
        Profit=('profit', 'sum'),
        Orders=('order_id', 'nunique')
    )
    .assign(
        Profit_Margin=lambda x: x['Profit'] / x['Sales']
    )
)
discount_profit

,Sales,Profit,Orders,Profit_Margin
discount_bin,,,,
0%,1087908.47,320987.88,2644,0.295050
1–20%,846522.17,100786.35,2507,0.119059
21–40%,234138.04,-35817.58,400,-0.152976
41–60%,71048.31,-28944.27,195,-0.407389
60%+,57584.08,-70614.59,594,-1.226287


In [421]:
discount_segment = (
    df
    .groupby(['segment', 'discount_bin'], observed=True)
    .agg(
        Sales=('sales', 'sum'),
        Profit=('profit', 'sum'),
        Orders=('order_id', 'nunique')
    )
    .assign(
        Profit_Margin=lambda x: x['Profit'] / x['Sales']
    )
)

discount_segment

Sales     Profit  Orders  Profit_Margin
segment     discount_bin                                             
Consumer    0%            532517.10  157901.96    1351       0.296520
            1–20%         439661.41   48107.77    1321       0.109420
            21–40%        132528.66  -16646.05     206      -0.125603
            41–60%         23971.09  -13541.47     106      -0.564908
            60%+           32723.08  -41702.88     313      -1.274418
Corporate   0%            358857.72  102150.95     784       0.284656
            1–20%         247915.75   29599.35     761       0.119393
            21–40%         67960.41  -13138.58     123      -0.193327
            41–60%         16826.16   -9026.02      55      -0.536428
            60%+           14586.40  -17606.25     180      -1.207032
Home Office 0%            196533.65   60934.97     509       0.310049
            1–20%         158945.01   23079.23     425       0.145203
            21–40%         33648.97   -6032.95      71      -0.179291
            41–60%         30251.06   -6376.78      34      -0.210795
            60%+           10274.60  -11305.46     101      -1.100331

In [422]:
region_metrics = (
    df
    .groupby('region')
    .agg(
        Sales=('sales', 'sum'),
        Profit=('profit', 'sum'),
        Orders=('order_id', 'nunique')
    )
    .assign(
        Profit_Margin=lambda x: x['Profit'] / x['Sales']
    )
    .sort_values('Profit', ascending=False)
)

region_metrics

,Sales,Profit,Orders,Profit_Margin
region,,,,
West,725457.93,108418.79,1611,0.149449
East,678781.36,91522.84,1401,0.134834
South,391721.90,46749.71,822,0.119344
Central,501239.88,39706.45,1175,0.079216


In [423]:
region_metrics.assign(
    Profit_Share=lambda x: x['Profit'] / x['Profit'].sum()
)

,Sales,Profit,Orders,Profit_Margin,Profit_Share
region,,,,,
West,725457.93,108418.79,1611,0.149449,0.378560
East,678781.36,91522.84,1401,0.134834,0.319565
South,391721.90,46749.71,822,0.119344,0.163233
Central,501239.88,39706.45,1175,0.079216,0.138641


In [424]:
discount_region = (
    df
    .groupby(['region', 'discount_bin'], observed=True)
    .agg(
        Sales=('sales', 'sum'),
        Profit=('profit', 'sum'),
        Orders=('order_id', 'nunique')
    )
    .assign(
        Profit_Margin=lambda x: x['Profit'] / x['Sales']
    )
    .reset_index()
)

discount_region

,region,discount_bin,Sales,Profit,Orders,Profit_Margin
0,Central,0%,243150.64,76125.45,410,0.313079
1,Central,1–20%,128955.12,15973.43,568,0.123868
2,Central,21–40%,98975.04,-11598.90,168,-0.117190
3,Central,41–60%,13195.40,-10254.40,138,-0.777119
4,Central,60%+,16963.68,-30539.13,250,-1.800266
5,East,0%,341742.54,105377.67,768,0.308354
6,East,1–20%,191168.24,28233.89,643,0.147691
7,East,21–40%,119266.92,-17871.03,216,-0.149841
8,East,41–60%,7308.47,-4255.83,18,-0.582315
9,East,60%+,19295.19,-19961.86,153,-1.034551


## Ключевые инсайты

### 1. Влияние скидок на маржинальность
- Во всех регионах наблюдается обратная зависимость между уровнем скидки и маржой прибыли.
- Критическим порогом являются скидки выше 20%, после которых маржинальность резко снижается.

### 2. Парадокс Центрального региона
- Центральный регион имеет высокий объём продаж, но минимальный вклад в общую прибыль.
- При отсутствии скидок Центральный регион демонстрирует наивысшую базовую маржинальность.
- Агрессивная дисконтная политика полностью нивелирует это преимущество.

### 3. Регион West как эталон
- Регион West является одним из ключевых драйверов выручки.
- В регионе отсутствуют скидки в диапазоне 21–40%, что коррелирует с более стабильной и высокой маржой.
- Практики региона West могут быть использованы как ориентир для других регионов.

## Рекомендации бизнесу

1. Ограничить использование скидок выше 20%
   Ввести дополнительный контроль и согласование для скидок свыше 20%, особенно в Центральном регионе.

2. Пересмотреть стратегию скидок в Центральном регионе
   Снизить долю средних и высоких скидок и протестировать стратегии продаж с фокусом на маржинальность.

3. Масштабировать успешные практики региона West
   Проанализировать и распространить ценовую и дисконтную стратегию региона West на другие регионы.